# Training the UNET

## Import

In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

## Seeding

In [2]:
os.environ["midterm"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Hyperparameters

In [3]:
height = 256
width = 256

batch_size = 16
lr = 2e-4
epochs = 100

## Path

In [4]:
dataset_path = "dataset"

files_dir = "files"
model_file = os.path.join(files_dir, "unet.h5")
log_file = os.path.join(files_dir, "log.csv")

## Creating Folder

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
create_dir(files_dir)

## Building UNET

### CBAM

In [7]:
def channel_attention_module(x, ratio=8):
    channel = x.shape[-1]
    
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)
    
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)
    
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)
    
    feats = x1 + x2
    feats = Activation("sigmoid")(feats)
    
    feats = Multiply()([x, feats])
    return feats

In [8]:
def spatial_attention_module(x):
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)
    
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    
    feats = Concatenate()([x1, x2])
    feats = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(feats)
    
    feats = Multiply()([x, feats])
    return feats

In [9]:
def cbam(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x

### Conv Block

In [10]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = cbam(x)
    
    return x

### Decoder Block

In [11]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

### UNET

In [12]:
def build_unet(input_shape):
    """ Inputs """
    inputs = Input(input_shape)
    
    """ ResNet50 Encoder """
    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)
    
    s1 = resnet50.get_layer("input_1").output
    s2 = resnet50.get_layer("conv1_relu").output
    s3 = resnet50.get_layer("conv2_block3_out").output
    s4 = resnet50.get_layer("conv3_block4_out").output
    
    """ Bridge """
    b1 = resnet50.get_layer("conv4_block6_out").output
    
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    return model

## Dataset Pipeline 

### Loading the training and validation dataset

In [13]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*")))
    
    valid_x = sorted(glob(os.path.join(path, "valid", "images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "masks", "*")))
    
    return (train_x, train_y), (valid_x, valid_y)

### Reading Images

In [14]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (width, height))
    x = x/255.0
    return x

### Reading Mask

In [15]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

### tf.data pipeline

In [16]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    
    return x, y

In [17]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

## Training

In [18]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 536 - 536
Valid: 67 - 67


In [19]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [20]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [21]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                               

#### Loss Function

In [22]:
def dice_loss(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + 1e-15) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-15)
    return 1.0 - dice

In [23]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss=dice_loss, optimizer=opt, metrics=["acc"], run_eagerly=True)

In [24]:
callbacks = [
        ModelCheckpoint(model_file, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10),
        CSVLogger(log_file),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

In [25]:
model.fit(
    train_dataset, 
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=callbacks,
)

Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.6469 - acc: 0.7802 
Epoch 1: val_loss improved from inf to 0.79571, saving model to files\unet.h5
34/34 [==============================] - 544s 16s/step - loss: 0.6469 - acc: 0.7802 - val_loss: 0.7957 - val_acc: 0.1296 - lr: 2.0000e-04
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.4812 - acc: 0.8335 
Epoch 2: val_loss did not improve from 0.79571
34/34 [==============================] - 513s 15s/step - loss: 0.4812 - acc: 0.8335 - val_loss: 0.7970 - val_acc: 0.8106 - lr: 2.0000e-04
Epoch 3/100
34/34 [==============================] - ETA: 0s - loss: 0.3547 - acc: 0.8554 
Epoch 3: val_loss did not improve from 0.79571
34/34 [==============================] - 501s 15s/step - loss: 0.3547 - acc: 0.8554 - val_loss: 0.7966 - val_acc: 0.8106 - lr: 2.0000e-04
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.2720 - acc: 0.8639 
Epoch 4: val_loss improved from 0.79571 to 0.79538